Imports:

In [2]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import random as r
import sklearn
from sklearn.utils.random import sample_without_replacement

random_state = 12

Convert 3d data to png images:

In [3]:
# Path to the directory containing your images and labels
images_dir = 'data/imagesTr'
labels_dir = 'data/labelsTr'
output_dir_images = 'data/images_png' 
output_dir_labels = 'data/labels_png'

os.makedirs(output_dir_labels, exist_ok=True)
os.makedirs(output_dir_images, exist_ok=True)

# List all the .nii.gz files in the directory
file_list = [file for file in os.listdir(images_dir) if file.endswith('.nii.gz') and file.startswith('lung')]
modified_list = [file_name[:-7] for file_name in file_list]
len_list = len(modified_list)
label_idx = 0

for idx, file in enumerate(file_list):
    print(f'\r{idx}/{len(file_list)}', end='')
    # Load the label
    label_path = os.path.join(labels_dir, file)
    label = nib.load(label_path).get_fdata()

    # Load the corresponding image
    image_path = os.path.join(images_dir, file)
    img = nib.load(image_path).get_fdata()

    labels_size = label.shape[2]
    indices = list(range(0, labels_size)) 

    # Convert slices with tumor labels to PNG
    for i in range(labels_size):  # Loop through the slices
        label_slice = label[:, :, i]
        ones = np.sum(label_slice == 1.0)

        slice_img = ((img[:, :, i] - np.min(img[:, :, i])) / (np.max(img[:, :, i]) - np.min(img[:, :, i]))) * 255.0
        slice_img = slice_img.astype(np.uint8)

        if ones > 0:
            # print("i", i)
            # print("indices", indices[i])
            # exit()
            indices.remove(i)

        if ones > 0.015 * label_slice.shape[0]:
            # Save the slice as PNG
            output_file_imgs = os.path.join(output_dir_images, ("image" + str(label_idx) + ".png"))
            output_file_labels = os.path.join(output_dir_labels, ("image" + str(label_idx) + ".png"))
            plt.imsave(output_file_imgs, slice_img)
            plt.imsave(output_file_labels, label_slice)

            label_idx = label_idx + 1

    
    selected_indices = r.sample(indices, 26)

    for j in selected_indices:
        label_slice = label[:, :, j]

        slice_img = ((img[:, :, j] - np.min(img[:, :, j])) / (np.max(img[:, :, j]) - np.min(img[:, :, j]))) * 255.0
        slice_img = slice_img.astype(np.uint8)            

        output_file_imgs = os.path.join(output_dir_images, ("image" + str(label_idx) + ".png"))
        output_file_labels = os.path.join(output_dir_labels, ("image" + str(label_idx) + ".png"))
        plt.imsave(output_file_imgs, slice_img)
        plt.imsave(output_file_labels, label_slice)

        label_idx = label_idx + 1

40/63